# 📈 Sentiment-Based Trading Strategy with Backtest

## Week 19: NLP & Alternative Data

This notebook implements a complete sentiment-based trading strategy with backtesting framework. We'll explore how to:

1. **Collect and process sentiment data** from news/social media
2. **Generate trading signals** based on sentiment indicators
3. **Implement a robust backtesting engine**
4. **Analyze strategy performance** with key metrics

### Strategy Overview

The core idea is simple: **market sentiment drives price movements**. When sentiment is overly positive, we go long; when sentiment turns negative, we exit or go short. We'll use sentiment scores combined with technical filters to generate trading signals.

---

## 1. Import Required Libraries

We'll use standard data science libraries along with `yfinance` for market data and `TextBlob` for sentiment analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 12

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 2. Load and Prepare Price Data

We'll download historical OHLCV data for a stock (AAPL) using Yahoo Finance. The data will be cleaned and prepared for backtesting.

In [2]:
# Configuration
TICKER = 'AAPL'
START_DATE = '2022-01-01'
END_DATE = '2024-12-31'

# Download price data
print(f"📊 Downloading {TICKER} data from {START_DATE} to {END_DATE}...")
price_data = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)

# Flatten multi-level columns if present
if isinstance(price_data.columns, pd.MultiIndex):
    price_data.columns = price_data.columns.get_level_values(0)

# Clean the data
price_data = price_data.dropna()
price_data.index = pd.to_datetime(price_data.index)

# Calculate returns
price_data['Returns'] = price_data['Close'].pct_change()
price_data['Log_Returns'] = np.log(price_data['Close'] / price_data['Close'].shift(1))

print(f"✅ Loaded {len(price_data)} trading days")
print(f"📅 Date range: {price_data.index[0].date()} to {price_data.index[-1].date()}")
price_data.head()

📊 Downloading AAPL data from 2022-01-01 to 2024-12-31...
✅ Loaded 752 trading days
📅 Date range: 2022-01-03 to 2024-12-30


Price,Close,High,Low,Open,Volume,Returns,Log_Returns
Date,,,,,,,
2022-01-03,178.270294,179.122429,174.058657,174.176186,104487900,NaN,NaN
2022-01-04,176.007782,179.181217,175.439697,178.877588,99310400,-0.012691,-0.012773
2022-01-05,171.326004,176.468135,171.051758,175.919643,94537600,-0.026600,-0.026960
2022-01-06,168.465988,171.698187,168.113384,169.151603,96904000,-0.016693,-0.016834
2022-01-07,168.632523,170.562047,167.515946,169.337730,86709100,0.000989,0.000988


## 3. Load and Process Sentiment Data

In production, sentiment data would come from:
- **News APIs** (NewsAPI, Bloomberg, Reuters)
- **Social Media** (Twitter/X, Reddit, StockTwits)
- **Earnings calls transcripts**
- **SEC filings**

For this educational example, we'll **simulate realistic sentiment data** that correlates with market movements.

In [ ]:
def generate_sentiment_data(price_df, noise_level=0.3, lag_days=1):
    """
    Generate simulated sentiment data that has some correlation with price movements.
    
    In reality, sentiment often LEADS price movements (predictive) or LAGS them (reactive).
    We simulate both components here.
    
    Parameters:
    -----------
    price_df : DataFrame with price data
    noise_level : float, amount of random noise (0-1)
    lag_days : int, sentiment lag relative to returns
    
    Returns:
    --------
    DataFrame with sentiment scores
    """
    np.random.seed(42)
    n = len(price_df)
    
    # Component 1: Sentiment that leads price (predictive)
    # Shifted forward returns (sentiment predicts future price)
    future_returns = price_df['Returns'].shift(-lag_days).fillna(0)
    predictive_sentiment = np.tanh(future_returns * 50)  # Scale and bound
    
    # Component 2: Sentiment that lags price (reactive)
    # Past returns affect current sentiment
    past_returns = price_df['Returns'].shift(lag_days).fillna(0)
    reactive_sentiment = np.tanh(past_returns * 30)
    
    # Component 3: Random noise
    noise = np.random.normal(0, noise_level, n)
    
    # Combine components
    raw_sentiment = 0.4 * predictive_sentiment + 0.3 * reactive_sentiment + 0.3 * noise
    
    # Normalize to [-1, 1]
    sentiment_score = np.clip(raw_sentiment, -1, 1)
    
    # Create sentiment DataFrame
    sentiment_df = pd.DataFrame({
        'Sentiment_Score': sentiment_score,
        'Sentiment_Volume': np.random.randint(50, 500, n),  # Number of articles/posts
        'Bullish_Ratio': (sentiment_score + 1) / 2,  # Convert to [0, 1]
    }, index=price_df.index)
    
    return sentiment_df

# Generate sentiment data
sentiment_data = generate_sentiment_data(price_data)

# Merge with price data
df = price_data.join(sentiment_data)
df = df.dropna()

print("📰 Sentiment Data Statistics:")
print(f"   Mean Sentiment: {df['Sentiment_Score'].mean():.4f}")
print(f"   Std Sentiment:  {df['Sentiment_Score'].std():.4f}")
print(f"   Min Sentiment:  {df['Sentiment_Score'].min():.4f}")
print(f"   Max Sentiment:  {df['Sentiment_Score'].max():.4f}")

df[['Close', 'Returns', 'Sentiment_Score', 'Sentiment_Volume', 'Bullish_Ratio']].head(10)

In [ ]:
# Visualize sentiment vs price
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price chart
axes[0].plot(df.index, df['Close'], 'b-', linewidth=1.5, label='Close Price')
axes[0].set_ylabel('Price ($)', fontsize=12)
axes[0].set_title(f'{TICKER} Price and Sentiment Analysis', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# Sentiment chart
colors = ['green' if x > 0 else 'red' for x in df['Sentiment_Score']]
axes[1].bar(df.index, df['Sentiment_Score'], color=colors, alpha=0.6, width=1)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].axhline(y=0.2, color='green', linestyle='--', alpha=0.5, label='Bullish Threshold')
axes[1].axhline(y=-0.2, color='red', linestyle='--', alpha=0.5, label='Bearish Threshold')
axes[1].set_ylabel('Sentiment Score', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Generate Trading Signals from Sentiment

We'll create trading signals using multiple sentiment-based indicators:

1. **Raw Sentiment Signal**: Buy when sentiment > threshold, sell when < -threshold
2. **Sentiment Momentum**: Track changes in sentiment direction
3. **Sentiment Moving Average Crossover**: Short-term vs long-term sentiment average
4. **Volume-Weighted Sentiment**: Weight sentiment by article/post volume

In [ ]:
class SentimentSignalGenerator:
    """
    Generate trading signals from sentiment data.
    """
    
    def __init__(self, df, sentiment_col='Sentiment_Score'):
        self.df = df.copy()
        self.sentiment_col = sentiment_col
        
    def add_sentiment_indicators(self, short_window=5, long_window=20):
        """Add technical indicators based on sentiment."""
        
        # Rolling sentiment averages
        self.df['Sentiment_SMA_Short'] = self.df[self.sentiment_col].rolling(short_window).mean()
        self.df['Sentiment_SMA_Long'] = self.df[self.sentiment_col].rolling(long_window).mean()
        
        # Sentiment momentum (rate of change)
        self.df['Sentiment_Momentum'] = self.df[self.sentiment_col].diff(5)
        
        # Sentiment volatility (uncertainty)
        self.df['Sentiment_Volatility'] = self.df[self.sentiment_col].rolling(10).std()
        
        # Volume-weighted sentiment
        self.df['Volume_Weighted_Sentiment'] = (
            self.df[self.sentiment_col] * self.df['Sentiment_Volume']
        ).rolling(5).sum() / self.df['Sentiment_Volume'].rolling(5).sum()
        
        # Extreme sentiment indicator
        sentiment_mean = self.df[self.sentiment_col].rolling(50).mean()
        sentiment_std = self.df[self.sentiment_col].rolling(50).std()
        self.df['Sentiment_ZScore'] = (self.df[self.sentiment_col] - sentiment_mean) / sentiment_std
        
        return self.df
    
    def generate_signals(self, strategy='combined', 
                         bullish_threshold=0.15, 
                         bearish_threshold=-0.15):
        """
        Generate trading signals based on sentiment strategy.
        
        Strategies:
        - 'threshold': Simple threshold-based signals
        - 'crossover': SMA crossover signals
        - 'momentum': Momentum-based signals
        - 'combined': Combination of multiple signals
        """
        
        self.df['Signal'] = 0  # 0 = no position, 1 = long, -1 = short
        
        if strategy == 'threshold':
            # Simple threshold strategy
            self.df.loc[self.df[self.sentiment_col] > bullish_threshold, 'Signal'] = 1
            self.df.loc[self.df[self.sentiment_col] < bearish_threshold, 'Signal'] = -1
            
        elif strategy == 'crossover':
            # SMA crossover strategy
            self.df['Signal'] = np.where(
                self.df['Sentiment_SMA_Short'] > self.df['Sentiment_SMA_Long'], 1,
                np.where(self.df['Sentiment_SMA_Short'] < self.df['Sentiment_SMA_Long'], -1, 0)
            )
            
        elif strategy == 'momentum':
            # Momentum strategy
            self.df.loc[self.df['Sentiment_Momentum'] > 0.05, 'Signal'] = 1
            self.df.loc[self.df['Sentiment_Momentum'] < -0.05, 'Signal'] = -1
            
        elif strategy == 'combined':
            # Combined strategy with multiple conditions
            conditions_long = (
                (self.df[self.sentiment_col] > bullish_threshold) &
                (self.df['Sentiment_SMA_Short'] > self.df['Sentiment_SMA_Long']) &
                (self.df['Sentiment_Momentum'] > 0)
            )
            
            conditions_short = (
                (self.df[self.sentiment_col] < bearish_threshold) &
                (self.df['Sentiment_SMA_Short'] < self.df['Sentiment_SMA_Long']) &
                (self.df['Sentiment_Momentum'] < 0)
            )
            
            self.df.loc[conditions_long, 'Signal'] = 1
            self.df.loc[conditions_short, 'Signal'] = -1
        
        # Calculate position changes
        self.df['Position'] = self.df['Signal'].shift(1).fillna(0)  # Enter next day
        self.df['Position_Change'] = self.df['Position'].diff().fillna(0)
        
        return self.df

# Initialize signal generator
signal_gen = SentimentSignalGenerator(df)

# Add indicators
df = signal_gen.add_sentiment_indicators(short_window=5, long_window=20)

# Generate signals using combined strategy
df = signal_gen.generate_signals(strategy='combined', 
                                  bullish_threshold=0.12, 
                                  bearish_threshold=-0.12)

# Summary of signals
print("📊 Signal Distribution:")
print(df['Signal'].value_counts())
print(f"\n📈 Total Trading Days: {len(df)}")
print(f"📊 Days in Long Position: {(df['Position'] == 1).sum()}")
print(f"📉 Days in Short Position: {(df['Position'] == -1).sum()}")
print(f"💤 Days in Cash: {(df['Position'] == 0).sum()}")

## 5. Define Position Sizing and Risk Management

Good risk management is crucial for any trading strategy. We'll implement:
- **Position Sizing**: Based on volatility (Kelly criterion inspired)
- **Stop-Loss**: Exit when price drops below threshold
- **Take-Profit**: Lock in gains at target
- **Maximum Drawdown Limit**: Stop trading if drawdown exceeds limit

In [ ]:
class RiskManager:
    """
    Risk management module for the trading strategy.
    """
    
    def __init__(self, 
                 max_position_size=1.0,
                 stop_loss_pct=0.05,
                 take_profit_pct=0.10,
                 max_drawdown_pct=0.20,
                 volatility_target=0.15):
        """
        Parameters:
        -----------
        max_position_size : float, maximum position as fraction of portfolio
        stop_loss_pct : float, stop loss percentage
        take_profit_pct : float, take profit percentage
        max_drawdown_pct : float, maximum allowed drawdown
        volatility_target : float, target annualized volatility
        """
        self.max_position_size = max_position_size
        self.stop_loss_pct = stop_loss_pct
        self.take_profit_pct = take_profit_pct
        self.max_drawdown_pct = max_drawdown_pct
        self.volatility_target = volatility_target
        
    def calculate_position_size(self, df, lookback=20):
        """
        Calculate position size based on volatility targeting.
        
        Position Size = (Target Vol / Realized Vol) * Base Position
        """
        # Calculate realized volatility (annualized)
        realized_vol = df['Returns'].rolling(lookback).std() * np.sqrt(252)
        
        # Calculate position size
        position_size = self.volatility_target / realized_vol
        
        # Cap at maximum position size
        position_size = position_size.clip(0, self.max_position_size)
        
        return position_size.fillna(self.max_position_size)
    
    def apply_risk_limits(self, df):
        """Apply stop-loss and take-profit levels."""
        
        df = df.copy()
        
        # Track entry prices and P&L
        df['Entry_Price'] = np.nan
        df['Trade_PnL'] = 0.0
        df['Risk_Adjusted_Position'] = df['Position'].copy()
        
        entry_price = None
        
        for i in range(1, len(df)):
            current_pos = df['Position'].iloc[i]
            prev_pos = df['Position'].iloc[i-1]
            current_price = df['Close'].iloc[i]
            
            # New position opened
            if current_pos != 0 and prev_pos == 0:
                entry_price = current_price
                df.iloc[i, df.columns.get_loc('Entry_Price')] = entry_price
                
            # Position still open
            elif current_pos != 0 and entry_price is not None:
                df.iloc[i, df.columns.get_loc('Entry_Price')] = entry_price
                
                # Calculate unrealized P&L
                if current_pos == 1:  # Long
                    pnl_pct = (current_price - entry_price) / entry_price
                else:  # Short
                    pnl_pct = (entry_price - current_price) / entry_price
                
                df.iloc[i, df.columns.get_loc('Trade_PnL')] = pnl_pct
                
                # Check stop-loss
                if pnl_pct <= -self.stop_loss_pct:
                    df.iloc[i, df.columns.get_loc('Risk_Adjusted_Position')] = 0
                    entry_price = None
                    
                # Check take-profit
                elif pnl_pct >= self.take_profit_pct:
                    df.iloc[i, df.columns.get_loc('Risk_Adjusted_Position')] = 0
                    entry_price = None
                    
            # Position closed
            elif current_pos == 0:
                entry_price = None
                
        return df

# Initialize risk manager
risk_manager = RiskManager(
    max_position_size=1.0,
    stop_loss_pct=0.05,      # 5% stop loss
    take_profit_pct=0.10,    # 10% take profit
    max_drawdown_pct=0.20,   # 20% max drawdown
    volatility_target=0.15   # 15% annualized vol target
)

# Calculate volatility-adjusted position sizes
df['Position_Size'] = risk_manager.calculate_position_size(df)

# Apply risk limits
df = risk_manager.apply_risk_limits(df)

print("🛡️ Risk Management Parameters:")
print(f"   Max Position Size: {risk_manager.max_position_size * 100}%")
print(f"   Stop Loss: {risk_manager.stop_loss_pct * 100}%")
print(f"   Take Profit: {risk_manager.take_profit_pct * 100}%")
print(f"   Max Drawdown Limit: {risk_manager.max_drawdown_pct * 100}%")
print(f"   Volatility Target: {risk_manager.volatility_target * 100}%")

## 6. Implement Backtest Engine

The backtest engine simulates the trading strategy on historical data, tracking:
- Portfolio value over time
- All trade entries and exits
- Transaction costs and slippage

In [ ]:
class BacktestEngine:
    """
    Vectorized backtesting engine for sentiment-based strategy.
    """
    
    def __init__(self, 
                 initial_capital=100000,
                 commission_pct=0.001,
                 slippage_pct=0.0005):
        """
        Parameters:
        -----------
        initial_capital : float, starting portfolio value
        commission_pct : float, commission as percentage of trade value
        slippage_pct : float, slippage as percentage of price
        """
        self.initial_capital = initial_capital
        self.commission_pct = commission_pct
        self.slippage_pct = slippage_pct
        self.results = None
        self.trades = []
        
    def run(self, df, position_col='Risk_Adjusted_Position', size_col='Position_Size'):
        """
        Run the backtest.
        
        Parameters:
        -----------
        df : DataFrame with signals and prices
        position_col : str, column name for position signals
        size_col : str, column name for position sizes
        
        Returns:
        --------
        DataFrame with backtest results
        """
        results = df.copy()
        
        # Calculate adjusted position with sizing
        results['Adjusted_Position'] = results[position_col] * results[size_col]
        
        # Calculate strategy returns (before costs)
        results['Strategy_Returns_Gross'] = results['Adjusted_Position'].shift(1) * results['Returns']
        
        # Calculate transaction costs
        position_changes = results['Adjusted_Position'].diff().abs()
        results['Transaction_Costs'] = position_changes * (self.commission_pct + self.slippage_pct)
        
        # Net strategy returns
        results['Strategy_Returns'] = results['Strategy_Returns_Gross'] - results['Transaction_Costs']
        
        # Calculate cumulative returns
        results['Strategy_Cumulative'] = (1 + results['Strategy_Returns'].fillna(0)).cumprod()
        results['Benchmark_Cumulative'] = (1 + results['Returns'].fillna(0)).cumprod()
        
        # Portfolio value
        results['Portfolio_Value'] = self.initial_capital * results['Strategy_Cumulative']
        results['Benchmark_Value'] = self.initial_capital * results['Benchmark_Cumulative']
        
        # Calculate drawdown
        results['Strategy_Peak'] = results['Portfolio_Value'].cummax()
        results['Drawdown'] = (results['Portfolio_Value'] - results['Strategy_Peak']) / results['Strategy_Peak']
        
        # Record trades
        self._record_trades(results, position_col)
        
        self.results = results
        return results
    
    def _record_trades(self, df, position_col):
        """Record individual trades for analysis."""
        self.trades = []
        
        entry_date = None
        entry_price = None
        entry_position = 0
        
        for i in range(1, len(df)):
            current_pos = df[position_col].iloc[i]
            prev_pos = df[position_col].iloc[i-1]
            
            # Trade entry
            if current_pos != 0 and prev_pos == 0:
                entry_date = df.index[i]
                entry_price = df['Close'].iloc[i]
                entry_position = current_pos
                
            # Trade exit
            elif current_pos == 0 and prev_pos != 0 and entry_date is not None:
                exit_date = df.index[i]
                exit_price = df['Close'].iloc[i]
                
                # Calculate P&L
                if entry_position == 1:  # Long
                    pnl_pct = (exit_price - entry_price) / entry_price
                else:  # Short
                    pnl_pct = (entry_price - exit_price) / entry_price
                
                trade = {
                    'Entry_Date': entry_date,
                    'Exit_Date': exit_date,
                    'Entry_Price': entry_price,
                    'Exit_Price': exit_price,
                    'Position': 'Long' if entry_position == 1 else 'Short',
                    'PnL_Pct': pnl_pct,
                    'Duration_Days': (exit_date - entry_date).days
                }
                self.trades.append(trade)
                
                entry_date = None
                entry_price = None
                
    def get_trades_df(self):
        """Return trades as DataFrame."""
        if self.trades:
            return pd.DataFrame(self.trades)
        return pd.DataFrame()

# Initialize and run backtest
backtest = BacktestEngine(
    initial_capital=100000,
    commission_pct=0.001,    # 0.1% commission
    slippage_pct=0.0005      # 0.05% slippage
)

# Run backtest
results = backtest.run(df)

# Get trades
trades_df = backtest.get_trades_df()

print(f"🔄 Backtest Complete!")
print(f"   Initial Capital: ${backtest.initial_capital:,.2f}")
print(f"   Final Portfolio Value: ${results['Portfolio_Value'].iloc[-1]:,.2f}")
print(f"   Benchmark Final Value: ${results['Benchmark_Value'].iloc[-1]:,.2f}")
print(f"\n📊 Total Trades: {len(trades_df)}")

if len(trades_df) > 0:
    print(f"\n📈 Trade Summary:")
    trades_df.head(10)

## 7. Calculate Performance Metrics

We'll calculate comprehensive performance metrics to evaluate our strategy:

In [ ]:
def calculate_performance_metrics(results, trades_df, risk_free_rate=0.04):
    """
    Calculate comprehensive performance metrics.
    
    Parameters:
    -----------
    results : DataFrame with backtest results
    trades_df : DataFrame with trade records
    risk_free_rate : float, annualized risk-free rate
    
    Returns:
    --------
    dict with performance metrics
    """
    metrics = {}
    
    # Basic returns
    strategy_returns = results['Strategy_Returns'].dropna()
    benchmark_returns = results['Returns'].dropna()
    
    # Total return
    metrics['Total_Return'] = results['Strategy_Cumulative'].iloc[-1] - 1
    metrics['Benchmark_Return'] = results['Benchmark_Cumulative'].iloc[-1] - 1
    metrics['Excess_Return'] = metrics['Total_Return'] - metrics['Benchmark_Return']
    
    # Annualized returns
    n_years = len(results) / 252
    metrics['Annualized_Return'] = (1 + metrics['Total_Return']) ** (1/n_years) - 1
    metrics['Benchmark_Annualized'] = (1 + metrics['Benchmark_Return']) ** (1/n_years) - 1
    
    # Volatility
    metrics['Annualized_Volatility'] = strategy_returns.std() * np.sqrt(252)
    metrics['Benchmark_Volatility'] = benchmark_returns.std() * np.sqrt(252)
    
    # Sharpe Ratio
    excess_returns = strategy_returns - risk_free_rate/252
    metrics['Sharpe_Ratio'] = np.sqrt(252) * excess_returns.mean() / strategy_returns.std() if strategy_returns.std() > 0 else 0
    
    # Sortino Ratio (downside deviation)
    downside_returns = strategy_returns[strategy_returns < 0]
    downside_std = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    metrics['Sortino_Ratio'] = (metrics['Annualized_Return'] - risk_free_rate) / downside_std if downside_std > 0 else 0
    
    # Maximum Drawdown
    metrics['Max_Drawdown'] = results['Drawdown'].min()
    
    # Calmar Ratio
    metrics['Calmar_Ratio'] = metrics['Annualized_Return'] / abs(metrics['Max_Drawdown']) if metrics['Max_Drawdown'] != 0 else 0
    
    # Beta (vs benchmark)
    if benchmark_returns.var() > 0:
        metrics['Beta'] = strategy_returns.cov(benchmark_returns) / benchmark_returns.var()
    else:
        metrics['Beta'] = 0
        
    # Alpha (Jensen's Alpha)
    metrics['Alpha'] = metrics['Annualized_Return'] - (risk_free_rate + metrics['Beta'] * (metrics['Benchmark_Annualized'] - risk_free_rate))
    
    # Information Ratio
    tracking_error = (strategy_returns - benchmark_returns).std() * np.sqrt(252)
    metrics['Information_Ratio'] = (metrics['Annualized_Return'] - metrics['Benchmark_Annualized']) / tracking_error if tracking_error > 0 else 0
    
    # Trade statistics
    if len(trades_df) > 0:
        metrics['Total_Trades'] = len(trades_df)
        metrics['Winning_Trades'] = (trades_df['PnL_Pct'] > 0).sum()
        metrics['Losing_Trades'] = (trades_df['PnL_Pct'] <= 0).sum()
        metrics['Win_Rate'] = metrics['Winning_Trades'] / metrics['Total_Trades']
        
        winning_trades = trades_df[trades_df['PnL_Pct'] > 0]['PnL_Pct']
        losing_trades = trades_df[trades_df['PnL_Pct'] <= 0]['PnL_Pct']
        
        metrics['Avg_Win'] = winning_trades.mean() if len(winning_trades) > 0 else 0
        metrics['Avg_Loss'] = losing_trades.mean() if len(losing_trades) > 0 else 0
        metrics['Profit_Factor'] = abs(winning_trades.sum() / losing_trades.sum()) if losing_trades.sum() != 0 else np.inf
        
        metrics['Avg_Trade_Duration'] = trades_df['Duration_Days'].mean()
        metrics['Max_Trade_Duration'] = trades_df['Duration_Days'].max()
    else:
        metrics['Total_Trades'] = 0
        metrics['Win_Rate'] = 0
        metrics['Profit_Factor'] = 0
    
    return metrics

# Calculate metrics
metrics = calculate_performance_metrics(results, trades_df)

# Display metrics
print("=" * 60)
print("📊 STRATEGY PERFORMANCE REPORT")
print("=" * 60)

print("\n📈 RETURNS")
print(f"   Total Return:        {metrics['Total_Return']*100:>10.2f}%")
print(f"   Benchmark Return:    {metrics['Benchmark_Return']*100:>10.2f}%")
print(f"   Excess Return:       {metrics['Excess_Return']*100:>10.2f}%")
print(f"   Annualized Return:   {metrics['Annualized_Return']*100:>10.2f}%")

print("\n📉 RISK")
print(f"   Annualized Vol:      {metrics['Annualized_Volatility']*100:>10.2f}%")
print(f"   Max Drawdown:        {metrics['Max_Drawdown']*100:>10.2f}%")
print(f"   Beta:                {metrics['Beta']:>10.2f}")

print("\n⚖️ RISK-ADJUSTED")
print(f"   Sharpe Ratio:        {metrics['Sharpe_Ratio']:>10.2f}")
print(f"   Sortino Ratio:       {metrics['Sortino_Ratio']:>10.2f}")
print(f"   Calmar Ratio:        {metrics['Calmar_Ratio']:>10.2f}")
print(f"   Information Ratio:   {metrics['Information_Ratio']:>10.2f}")
print(f"   Alpha (annualized):  {metrics['Alpha']*100:>10.2f}%")

print("\n🔄 TRADE STATISTICS")
print(f"   Total Trades:        {metrics['Total_Trades']:>10}")
if metrics['Total_Trades'] > 0:
    print(f"   Win Rate:            {metrics['Win_Rate']*100:>10.1f}%")
    print(f"   Profit Factor:       {metrics['Profit_Factor']:>10.2f}")
    print(f"   Avg Win:             {metrics['Avg_Win']*100:>10.2f}%")
    print(f"   Avg Loss:            {metrics['Avg_Loss']*100:>10.2f}%")
    print(f"   Avg Trade Duration:  {metrics['Avg_Trade_Duration']:>10.1f} days")

print("\n" + "=" * 60)

## 8. Visualize Strategy Results

Let's create comprehensive visualizations of our strategy performance.

In [ ]:
def plot_strategy_results(results, trades_df, metrics):
    """Create comprehensive visualization of strategy results."""
    
    fig = plt.figure(figsize=(16, 14))
    
    # 1. Equity Curve
    ax1 = fig.add_subplot(3, 2, 1)
    ax1.plot(results.index, results['Portfolio_Value'] / 1000, 'b-', 
             linewidth=2, label='Strategy')
    ax1.plot(results.index, results['Benchmark_Value'] / 1000, 'gray', 
             linewidth=1.5, alpha=0.7, label='Buy & Hold')
    ax1.fill_between(results.index, results['Portfolio_Value'] / 1000, 
                     results['Benchmark_Value'] / 1000, 
                     where=results['Portfolio_Value'] > results['Benchmark_Value'],
                     color='green', alpha=0.2)
    ax1.fill_between(results.index, results['Portfolio_Value'] / 1000, 
                     results['Benchmark_Value'] / 1000,
                     where=results['Portfolio_Value'] <= results['Benchmark_Value'],
                     color='red', alpha=0.2)
    ax1.set_title('📈 Equity Curve: Strategy vs Benchmark', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($K)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # 2. Drawdown Chart
    ax2 = fig.add_subplot(3, 2, 2)
    ax2.fill_between(results.index, results['Drawdown'] * 100, 0, 
                     color='red', alpha=0.5)
    ax2.axhline(y=metrics['Max_Drawdown'] * 100, color='darkred', 
                linestyle='--', label=f'Max DD: {metrics["Max_Drawdown"]*100:.1f}%')
    ax2.set_title('📉 Strategy Drawdown', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Drawdown (%)')
    ax2.legend(loc='lower left')
    ax2.grid(True, alpha=0.3)
    
    # 3. Rolling Sharpe Ratio
    ax3 = fig.add_subplot(3, 2, 3)
    rolling_sharpe = results['Strategy_Returns'].rolling(63).apply(
        lambda x: np.sqrt(252) * x.mean() / x.std() if x.std() > 0 else 0
    )
    ax3.plot(results.index, rolling_sharpe, 'purple', linewidth=1.5)
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax3.axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Good (1.0)')
    ax3.axhline(y=2, color='darkgreen', linestyle='--', alpha=0.5, label='Excellent (2.0)')
    ax3.set_title('📊 Rolling Sharpe Ratio (63-day)', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Sharpe Ratio')
    ax3.legend(loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    # 4. Monthly Returns Heatmap
    ax4 = fig.add_subplot(3, 2, 4)
    monthly_returns = results['Strategy_Returns'].resample('M').apply(
        lambda x: (1 + x).prod() - 1
    )
    monthly_df = pd.DataFrame({
        'Year': monthly_returns.index.year,
        'Month': monthly_returns.index.month,
        'Return': monthly_returns.values * 100
    })
    monthly_pivot = monthly_df.pivot(index='Year', columns='Month', values='Return')
    
    # Simple color-coded bar chart for monthly returns
    monthly_returns_pct = monthly_returns * 100
    colors = ['green' if r > 0 else 'red' for r in monthly_returns_pct]
    ax4.bar(range(len(monthly_returns_pct)), monthly_returns_pct, color=colors, alpha=0.7)
    ax4.axhline(y=0, color='black', linewidth=0.5)
    ax4.set_title('📅 Monthly Returns (%)', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Return (%)')
    ax4.set_xlabel('Month')
    ax4.grid(True, alpha=0.3)
    
    # 5. Trade Entry/Exit Points on Price Chart
    ax5 = fig.add_subplot(3, 2, 5)
    ax5.plot(results.index, results['Close'], 'k-', linewidth=1, alpha=0.7, label='Price')
    
    # Mark long entries/exits
    if len(trades_df) > 0:
        long_trades = trades_df[trades_df['Position'] == 'Long']
        short_trades = trades_df[trades_df['Position'] == 'Short']
        
        for _, trade in long_trades.iterrows():
            ax5.scatter(trade['Entry_Date'], trade['Entry_Price'], 
                       marker='^', color='green', s=100, zorder=5)
            ax5.scatter(trade['Exit_Date'], trade['Exit_Price'], 
                       marker='v', color='blue', s=100, zorder=5)
            
        for _, trade in short_trades.iterrows():
            ax5.scatter(trade['Entry_Date'], trade['Entry_Price'], 
                       marker='v', color='red', s=100, zorder=5)
            ax5.scatter(trade['Exit_Date'], trade['Exit_Price'], 
                       marker='^', color='orange', s=100, zorder=5)
    
    ax5.scatter([], [], marker='^', color='green', s=100, label='Long Entry')
    ax5.scatter([], [], marker='v', color='blue', s=100, label='Long Exit')
    ax5.scatter([], [], marker='v', color='red', s=100, label='Short Entry')
    ax5.scatter([], [], marker='^', color='orange', s=100, label='Short Exit')
    ax5.set_title('🎯 Trade Entry/Exit Points', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Price ($)')
    ax5.legend(loc='upper left', fontsize=8)
    ax5.grid(True, alpha=0.3)
    
    # 6. Win/Loss Distribution
    ax6 = fig.add_subplot(3, 2, 6)
    if len(trades_df) > 0:
        trade_returns = trades_df['PnL_Pct'] * 100
        colors = ['green' if r > 0 else 'red' for r in trade_returns]
        ax6.bar(range(len(trade_returns)), trade_returns, color=colors, alpha=0.7)
        ax6.axhline(y=0, color='black', linewidth=0.5)
        ax6.axhline(y=trade_returns.mean(), color='blue', linestyle='--', 
                   label=f'Avg: {trade_returns.mean():.2f}%')
        ax6.set_title('🎲 Individual Trade Returns', fontsize=12, fontweight='bold')
        ax6.set_ylabel('Return (%)')
        ax6.set_xlabel('Trade #')
        ax6.legend(loc='upper right')
    else:
        ax6.text(0.5, 0.5, 'No trades recorded', ha='center', va='center', fontsize=14)
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot results
plot_strategy_results(results, trades_df, metrics)

## 9. Strategy Analysis & Sensitivity

Let's analyze how the strategy performs under different parameter settings.

In [ ]:
def run_sensitivity_analysis(base_df, thresholds, strategies):
    """
    Run sensitivity analysis across different parameters.
    """
    sensitivity_results = []
    
    for strategy in strategies:
        for thresh in thresholds:
            # Create fresh copy
            test_df = base_df.copy()
            
            # Generate signals
            sig_gen = SentimentSignalGenerator(test_df)
            test_df = sig_gen.add_sentiment_indicators()
            test_df = sig_gen.generate_signals(
                strategy=strategy,
                bullish_threshold=thresh,
                bearish_threshold=-thresh
            )
            
            # Apply risk management
            rm = RiskManager()
            test_df['Position_Size'] = rm.calculate_position_size(test_df)
            test_df = rm.apply_risk_limits(test_df)
            
            # Run backtest
            bt = BacktestEngine()
            test_results = bt.run(test_df)
            trades = bt.get_trades_df()
            
            # Calculate metrics
            test_metrics = calculate_performance_metrics(test_results, trades)
            
            sensitivity_results.append({
                'Strategy': strategy,
                'Threshold': thresh,
                'Total_Return': test_metrics['Total_Return'],
                'Sharpe_Ratio': test_metrics['Sharpe_Ratio'],
                'Max_Drawdown': test_metrics['Max_Drawdown'],
                'Total_Trades': test_metrics['Total_Trades'],
                'Win_Rate': test_metrics.get('Win_Rate', 0)
            })
    
    return pd.DataFrame(sensitivity_results)

# Run sensitivity analysis
thresholds = [0.05, 0.10, 0.15, 0.20, 0.25]
strategies = ['threshold', 'crossover', 'combined']

print("🔬 Running sensitivity analysis...")
sensitivity_df = run_sensitivity_analysis(df.copy(), thresholds, strategies)

# Pivot table for visualization
print("\n📊 Sharpe Ratio by Strategy and Threshold:")
sharpe_pivot = sensitivity_df.pivot(index='Threshold', columns='Strategy', values='Sharpe_Ratio')
print(sharpe_pivot.round(2))

print("\n📈 Total Return by Strategy and Threshold:")
return_pivot = sensitivity_df.pivot(index='Threshold', columns='Strategy', values='Total_Return')
print((return_pivot * 100).round(2).astype(str) + '%')

In [ ]:
# Visualize sensitivity analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sharpe Ratio by threshold
ax1 = axes[0, 0]
for strategy in strategies:
    data = sensitivity_df[sensitivity_df['Strategy'] == strategy]
    ax1.plot(data['Threshold'], data['Sharpe_Ratio'], 'o-', linewidth=2, 
             markersize=8, label=strategy.capitalize())
ax1.axhline(y=1, color='gray', linestyle='--', alpha=0.5)
ax1.set_xlabel('Sentiment Threshold')
ax1.set_ylabel('Sharpe Ratio')
ax1.set_title('⚖️ Sharpe Ratio vs Threshold', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Total Return by threshold
ax2 = axes[0, 1]
for strategy in strategies:
    data = sensitivity_df[sensitivity_df['Strategy'] == strategy]
    ax2.plot(data['Threshold'], data['Total_Return'] * 100, 's-', linewidth=2, 
             markersize=8, label=strategy.capitalize())
ax2.axhline(y=metrics['Benchmark_Return'] * 100, color='gray', linestyle='--', 
            alpha=0.5, label='Benchmark')
ax2.set_xlabel('Sentiment Threshold')
ax2.set_ylabel('Total Return (%)')
ax2.set_title('📈 Total Return vs Threshold', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Max Drawdown by threshold
ax3 = axes[1, 0]
for strategy in strategies:
    data = sensitivity_df[sensitivity_df['Strategy'] == strategy]
    ax3.plot(data['Threshold'], data['Max_Drawdown'] * 100, '^-', linewidth=2, 
             markersize=8, label=strategy.capitalize())
ax3.set_xlabel('Sentiment Threshold')
ax3.set_ylabel('Max Drawdown (%)')
ax3.set_title('📉 Max Drawdown vs Threshold', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Number of Trades by threshold
ax4 = axes[1, 1]
for strategy in strategies:
    data = sensitivity_df[sensitivity_df['Strategy'] == strategy]
    ax4.bar([x + strategies.index(strategy) * 0.25 for x in range(len(thresholds))], 
            data['Total_Trades'], width=0.25, label=strategy.capitalize(), alpha=0.7)
ax4.set_xticks([x + 0.25 for x in range(len(thresholds))])
ax4.set_xticklabels([str(t) for t in thresholds])
ax4.set_xlabel('Sentiment Threshold')
ax4.set_ylabel('Number of Trades')
ax4.set_title('🔄 Trade Frequency vs Threshold', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Conclusions & Key Takeaways

### 📚 What We Learned

1. **Sentiment-Based Trading**: Market sentiment from news and social media can provide valuable trading signals, especially when combined with technical filters.

2. **Signal Generation**: Multiple approaches exist:
   - Simple threshold strategies
   - Moving average crossovers on sentiment
   - Momentum-based signals
   - Combined multi-factor approaches

3. **Risk Management is Crucial**: Position sizing, stop-losses, and drawdown limits significantly impact strategy performance.

4. **Backtesting Considerations**:
   - Include transaction costs and slippage
   - Use out-of-sample testing
   - Be aware of look-ahead bias
   - Simulated sentiment != real sentiment (our data is artificially correlated)

### ⚠️ Important Caveats

- **Simulated Data**: This notebook uses simulated sentiment data. Real sentiment data:
  - Is noisy and often contradictory
  - Has different characteristics across sources
  - Requires significant cleaning and normalization
  - May have survivorship bias

- **Overfitting Risk**: Optimizing parameters on historical data can lead to overfitting. Always use:
  - Walk-forward optimization
  - Out-of-sample testing
  - Cross-validation across different time periods

- **Market Regime Dependence**: Sentiment strategies may work well in some market regimes and poorly in others.

### 🚀 Next Steps

1. **Real Sentiment Data**: Integrate with APIs like NewsAPI, Twitter, or financial news providers
2. **NLP Pipeline**: Build custom sentiment models using BERT, FinBERT, or fine-tuned transformers
3. **Multi-Asset**: Extend to multiple assets with correlation-aware position sizing
4. **Machine Learning**: Use ML models to combine sentiment with other features
5. **Live Trading**: Implement paper trading to validate strategy in real-time

In [ ]:
# Final Summary
print("=" * 70)
print("🏁 SENTIMENT TRADING STRATEGY - FINAL SUMMARY")
print("=" * 70)
print(f"""
Asset:              {TICKER}
Period:             {START_DATE} to {END_DATE}
Initial Capital:    ${backtest.initial_capital:,.2f}

STRATEGY PERFORMANCE:
---------------------
Total Return:       {metrics['Total_Return']*100:.2f}%
Benchmark Return:   {metrics['Benchmark_Return']*100:.2f}%
Alpha:              {metrics['Alpha']*100:.2f}%
Sharpe Ratio:       {metrics['Sharpe_Ratio']:.2f}
Max Drawdown:       {metrics['Max_Drawdown']*100:.2f}%

TRADING ACTIVITY:
-----------------
Total Trades:       {metrics['Total_Trades']}
Win Rate:           {metrics.get('Win_Rate', 0)*100:.1f}%
Profit Factor:      {metrics.get('Profit_Factor', 0):.2f}

CONCLUSION:
-----------
The sentiment-based trading strategy {"outperformed" if metrics['Excess_Return'] > 0 else "underperformed"} 
the buy-and-hold benchmark by {abs(metrics['Excess_Return'])*100:.2f}%.
""")
print("=" * 70)